In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import sys
import natsort
import GrowthAssayPlotlib

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
ga1_file = 'GA1 parasitemia.xlsx'
column_title_row = 3
OUTPUT_FOLDER = ''

df1 = GrowthAssayPlotlib.load_file(ga1_file, column_title_row)
df1_melted_replicas = GrowthAssayPlotlib.melt_df(df1, id_vars=['Plate number', 'Well position', 'Line', 'Clone', 'Strain', 'Speed'])

df1_melted_replicas.head()

In [ ]:
# plot all 
MAX_INVASION_RATE_PLOT = 10
output_folder_ga1 = GrowthAssayPlotlib.get_output_folder('GA1', OUTPUT_FOLDER)

df1_melted_replicas_filtered = GrowthAssayPlotlib.filter_dataframe(
    df=df1_melted_replicas,
    ignore_repeats=[
        # 'Repeat 1',
    ],
    filter_list_filename='GA1 filtered',
)

# group replicas per repeat
df1_melted = GrowthAssayPlotlib.group_repeats(
    df_melted_replicas_filtered = df1_melted_replicas_filtered,
    group_keys=['Line', 'Speed', 'Repeat', 'Strain', 'Clone'],
)

df1_melted.to_json('GA1_df_melted.json')

# GrowthAssayPlotlib.plot_repeats_for_line(df1_melted_replicas_filtered, MAX_INVASION_RATE_PLOT, output_folder_ga1)
# GrowthAssayPlotlib.plot_repeats_for_wells(df1_melted_replicas_filtered, MAX_INVASION_RATE_PLOT, output_folder_ga1)
# GrowthAssayPlotlib.plot_parasimetia_vs_invasion_rate(df1_melted_replicas, ['Line', 'Speed'], output_folder=output_folder_ga1)
# for label, df_plot in [('with replicas', df1_melted_replicas), ('averaged replicas', df1_melted)]:
#     GrowthAssayPlotlib.plot_line(df_plot, label, MAX_INVASION_RATE_PLOT, output_folder_ga1)
#     GrowthAssayPlotlib.plot_speed(df_plot, label, MAX_INVASION_RATE_PLOT, output_folder_ga1)

In [ ]:
df1_melted_replicas_filtered

In [ ]:

remove_lines = ['KOEBA175', 'Barseq']
df1_melted_q = df1_melted_replicas_filtered.query('Strain != @remove_lines')



GrowthAssayPlotlib.plot_pretty_barplot(
    df=df1_melted_q,
    plot_clones=False, # or 'Line' if you want clones to show separatley
    output_folder=output_folder_ga1,
)
GrowthAssayPlotlib.plot_pretty_barplot(
    df=df1_melted_q,
    plot_clones=True, # or 'Line' if you want clones to show separatley
    output_folder=output_folder_ga1,
)

plt.xticks(rotation=90)

clone_list = [
    'NF54',
    'KOEBA181 C1',
    'KORH4 C1',
    'KOEBA140 C4', 
    'KOEBA175 C6',
    'KORH2a C3',
    'KORH1 C1',
    'KOPfs25 C1',
    'KOP230P C3',
]
dfq = df1_melted_q.query('Line == @clone_list')
GrowthAssayPlotlib.plot_pretty_barplot(
    df=dfq,
    plot_clones=True, # or 'Line' if you want clones to show separatley
    output_folder=output_folder_ga1,
    bar_width=0.4,
)

plt.gcf().set_size_inches(15,9)
plt.xticks(rotation=90)
plt.ylim((0,12))

plt.rcParams['svg.fonttype'] = 'none'
plt.savefig(f'Graphs GA1/clones_separate_fig5.svg', bbox_inches='tight')


In [ ]:
GrowthAssayPlotlib.significance_testing(
    df=df1_melted, 
    group_keys=['Line', 'Speed'],
    wildtype='KOPfs25 C1'
)

In [ ]:
df1_melted

In [ ]:
dfp = df1_melted.groupby(['Line', 'Speed'], as_index=False).agg(
    mean=('Invasion rate', 'mean'),
    std=('Invasion rate', 'std'),
    count=('Invasion rate', 'count'),
)
dfp['ste'] = dfp['std']/np.sqrt(dfp['count'])
dfp